# Creating a simple bot. 

See following for more details on this. 

### Before you start this notbook. 
Create a .env files in same place you have placed this notebook and add following two keys. You can get the API keys from respective website.  
OPENAI_API_KEY=<<key>>  
TAVILY_API_KEY=<<key>>

In [ ]:
from dotenv import load_dotenv
import os
import httpx
import certifi
import ssl
import json


ssl_context = ssl.create_default_context(cafile=certifi.where())
os.environ["SSL_CERT_FILE"] = certifi.where() # was getting a cert error with openAI. So had to do this.
_ = load_dotenv() # this should load the require keys for Open AI.Key should be OPENAI_API_KEY
from openai import OpenAI

In [2]:
# if you want to view what all env variables are set
# for k,v in os.environ.items():
#     print(k,v)

In [3]:
client = OpenAI()

In [4]:
def weather_call(zip_code):
    return f"Weather for {zip_code} is sunny today if you want to go to the store"


def order_status(name):
    print(f"Pulling orders status for: {name}")
    if name in "1234599": 
        return(f"Order status for {name} at zip 60502 is ready for pickup")
    
    elif name in "123455":
        return("Order is in progress and will be delivered to 60502 soon")
    elif name in "123465":
        return(f"Order status for {name} at zip  is ready for pickup")
    elif name in "123459":
        return(f"Order status for {name} is ready to be delivered to 60502 soon")
    else:
        return("Unknown at this time.")

known_actions = {
    "weather_call": weather_call,
    "order_status": order_status
}

In [5]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.Any other request you get you politely says sorry and can't be processed

Your available actions are:

order_status:
e.g. order_status: 123456
Runs a order_status and returns the number - Order status for the order # 12345 for Zip 60502 is Ready for Pickup.

weather_call:
e.g. weather_call(60502)
returns Weather for that location

Example session:

Question: What is the status of the order 12345?
Thought: I should lookup the order status in order_status
Action: order_status(12345)
PAUSE

You will be called again with this:

Observation: Order status for 12345 at zip 60502 is ready for pickup

However if you see a Zip code with ready for pickup status you make another Action to get weather information.You can make this call in parallel or in sequence.

Question: What is the status of the order 12345?
Thought: I should lookup the order status in order_status
Action: order_status(12345)
PAUSE
Observation: Order status for 12345 at zip 60502 is ready for pickup
Thought: I should lookup the weather for 60502
Action: weather_call(60502)
PAUSE

You then output:

Answer: Order status for 12345 at zip 60502 is ready for pickup and the weather details.
If there is any missing information from the tool you just respond back with the last available information from the tool. 
""".strip()

In [6]:
initial_context_set = [{"role": "system", "content": prompt}]

In [7]:
# Creating a chat with openAI.
# Temperature set to 0 to get deterministic results. So the mode is less creative in response.
# initializing the chat
completion = client.chat.completions.create(
                        model="gpt-4o", 
                        temperature=0,
                        messages=initial_context_set)

**Now imagin a customer is asking for a status on their `order#12345`**  
You send this message to LLM to identify next steps to answer and respond to the customer   

In [8]:
message_to_llm = [
    {"role": "system", "content": prompt},
    {"role": "user", "content": "What is the status of the order 1234599" }
]
completion = client.chat.completions.create(
                        model="gpt-4o", 
                        temperature=0,
                        messages=message_to_llm)

In [9]:
completion.choices[0].message

ChatCompletionMessage(content='Thought: I should lookup the order status in order_status\nAction: order_status(1234599)\nPAUSE', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, annotations=[])

**Here is the response from the LLM model**   
As you can usee there is an Action to be performed before we can respond to the customer. So to get the final response from the LLM model it needs additional details

In [10]:
print(completion.choices[0].message.content)

Thought: I should lookup the order status in order_status
Action: order_status(1234599)
PAUSE


Performing the action **manually** to demonostrate the flow. 
`Action: order_status(1234599)`

In [11]:
order_status_resp = order_status("1234599")
print(f"Response from order stauts call: {order_status_resp}")

Pulling orders status for: 1234599
Response from order stauts call: Order status for 1234599 at zip 60502 is ready for pickup


**Prepare the response we got from order status to send it to the model again**

In [12]:
response_to_llm_next_prompt = "Observation: {}".format(order_status_resp)
response_to_llm_next_prompt

'Observation: Order status for 1234599 at zip 60502 is ready for pickup'

Prepare the above input to the LLM

In [13]:
message_to_llm = [
    {"role": "system", "content": prompt},
    {"role": "user", "content": "What is the status of the order 1234599" },
     {"role": "user", "content": response_to_llm_next_prompt }
]
completion = client.chat.completions.create(
                        model="gpt-4o", 
                        temperature=0,
                        messages=message_to_llm)

In [14]:
print(completion.choices[0].message.content)

Thought: I should lookup the weather for 60502
Action: weather_call(60502)
PAUSE


As you can see above LLM discovered that there is a zip code in the inputs to it. So it is asking for a weather to finalize the response to customer

**Get weather information and prepare the response back to LLM**

In [39]:
weather_call = weather_call("60502")
weather_call

'Weather for 60502 is sunny today'

In [42]:
response_weather_to_llm_next_prompt = "Observation: {}".format(weather_call)
response_weather_to_llm_next_prompt

'Observation: Weather for 60502 is sunny today'

In [43]:
message_to_llm = [
    {"role": "system", "content": prompt},
    {"role": "user", "content": "What is the status of the order 1234599" },
     {"role": "user", "content": response_to_llm_next_prompt },
    {"role": "user", "content": response_weather_to_llm_next_prompt }
]
completion = client.chat.completions.create(
                        model="gpt-4o", 
                        temperature=0,
                        messages=message_to_llm)

### Final response to the customer

In [44]:
print(completion.choices[0].message.content)

Answer: Order status for 12345 at zip 60502 is ready for pickup and the weather is sunny today.


## A scenario where cusomer does not provide an order number

In [46]:
message_to_llm = [
    {"role": "system", "content": prompt},
    {"role": "user", "content": "What is the status of the order" }
]
completion = client.chat.completions.create(
                        model="gpt-4o", 
                        temperature=0,
                        messages=message_to_llm)

In [47]:
print(completion.choices[0].message.content)

Sorry, I can't process that request without an order number. Please provide the order number to check the status.


## A scenario where there is no zip available

In [65]:
message_to_llm = [
    {"role": "system", "content": prompt},
    {"role": "user", "content": "What is the status of the order 123465" }
]
completion = client.chat.completions.create(
                        model="gpt-4o", 
                        temperature=0,
                        messages=message_to_llm)

In [66]:
order_status_resp = order_status("123465")
print(f"Response from order stauts call: {order_status_resp}")

Response from order stauts call: Order status for 12345 at zip  is ready for pickup


In [67]:
response_to_llm_next_prompt = "Observation: {}".format(order_status_resp)
response_to_llm_next_prompt

'Observation: Order status for 12345 at zip  is ready for pickup'

In [68]:
message_to_llm = [
    {"role": "system", "content": prompt},
    {"role": "user", "content": "What is the status of the order 123465" },
     {"role": "user", "content": response_to_llm_next_prompt }
]
completion = client.chat.completions.create(
                        model="gpt-4o", 
                        temperature=0,
                        messages=message_to_llm)

### Final response to the customer

In [69]:
print(completion.choices[0].message.content)

Answer: Order status for 12345 is ready for pickup.


# Using Tool (better approach)

In [214]:

client2 = OpenAI()

tools = [{
    "type": "function",
    "function": {
        "name": "weather_call",
        "description": "Get current temperature for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "zip_code": {
                    "type": "string",
                    "description": "Zip code"
                }
            },
            "required": [
                "zip_code"
            ],
            "additionalProperties": False
        },
        "strict": True
    }
},{
    "type": "function",
    "function": {
        "name": "order_status",
        "description": "Get current order status.",
        "parameters": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "Oorder id "
                }
            },
            "required": [
                "order_id"
            ],
            "additionalProperties": False
        },
        "strict": True
    }
}
         
        ]



In [215]:
prompt_for_tooling = """
You are retial ecom bot agent. You are capabale of giving order status info along with weather based on the order zip availability. 
Order Zip will be returned by order_status and not consider the zip code provided by users.
Any other request you get you politely says sorry and can't be processed

Your available function are:

order_status

weather_call

You then output:
Order status for 12345 at zip 60502 is ready for pickup and the weather details are...

If there is any missing information from the tool you just respond back with the last available information from the tool. 
""".strip()

In [216]:
messages = [
    {"role": "system", "content": prompt_for_tooling},
     {"role": "user", "content": "What is the status of the order 1234599"}
]

In [219]:
completion = client2.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools
)



[ChatCompletionMessageToolCall(id='call_5itmdTYu72PJxibkOb4vyQz8', function=Function(arguments='{"order_id":"1234599"}', name='order_status'), type='function')]


In [250]:
print(completion)

ChatCompletion(id='chatcmpl-BA1Grm4JWGuP46tjvvnyEIB56lYTD', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_5itmdTYu72PJxibkOb4vyQz8', function=Function(arguments='{"order_id":"1234599"}', name='order_status'), type='function')], annotations=[]))], created=1741727865, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_eb9dce56a8', usage=CompletionUsage(completion_tokens=18, prompt_tokens=206, total_tokens=224, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [237]:
def get_tool_response(completion):
    assistant_message = completion.choices[0].message
    
    # Convert to dictionary
    assistant_dict = {
        "role": assistant_message.role,
        "content": assistant_message.content,  # Will be None in tool calls
        "tool_calls": [
            {
                "id": tool_call.id,
                "type": tool_call.type,
                "function": {
                    "name": tool_call.function.name,
                    "arguments": tool_call.function.arguments
                }
            }
            for tool_call in (assistant_message.tool_calls or [])
        ]
    }
    return assistant_dict

In [239]:
assistant_dict = get_tool_response(completion)
assistant_dict

{'role': 'assistant',
 'content': None,
 'tool_calls': [{'id': 'call_5itmdTYu72PJxibkOb4vyQz8',
   'type': 'function',
   'function': {'name': 'order_status',
    'arguments': '{"order_id":"1234599"}'}}]}

In [222]:
tool_call = completion.choices[0].message.tool_calls[0]
args = json.loads(tool_call.function.arguments)

In [223]:
tool_call

ChatCompletionMessageToolCall(id='call_5itmdTYu72PJxibkOb4vyQz8', function=Function(arguments='{"order_id":"1234599"}', name='order_status'), type='function')

In [224]:
args

{'order_id': '1234599'}

**Now we call the tools based on the above response from LLM**

In [225]:
tool_call.function.name

'order_status'

In [226]:
tool_call.id

'call_5itmdTYu72PJxibkOb4vyQz8'

In [227]:
args["order_id"]

'1234599'

In [228]:
result = eval(tool_call.function.name)(args["order_id"])

Pulling orders status for: 1234599


In [229]:
result

'Order status for 1234599 at zip 60502 is ready for pickup'

In [230]:
print(f"Tool Call ID: {tool_call.id}") 

Tool Call ID: call_5itmdTYu72PJxibkOb4vyQz8


In [232]:
messages

[{'role': 'system',
  'content': "You are retial ecom bot agent. You are capabale of giving order status info along with weather based on the order zip availability. \nOrder Zip will be returned by order_status and not consider the zip code provided by users.\nAny other request you get you politely says sorry and can't be processed\n\nYour available function are:\n\norder_status\n\nweather_call\n\nYou then output:\nOrder status for 12345 at zip 60502 is ready for pickup and the weather details are...\n\nIf there is any missing information from the tool you just respond back with the last available information from the tool."},
 {'role': 'user', 'content': 'What is the status of the order 1234599'}]

In [233]:
messages.append(assistant_dict)

In [234]:
messages.append(
    {"role": "tool", "content": str(result), "tool_call_id": tool_call.id}
)

In [235]:
messages

[{'role': 'system',
  'content': "You are retial ecom bot agent. You are capabale of giving order status info along with weather based on the order zip availability. \nOrder Zip will be returned by order_status and not consider the zip code provided by users.\nAny other request you get you politely says sorry and can't be processed\n\nYour available function are:\n\norder_status\n\nweather_call\n\nYou then output:\nOrder status for 12345 at zip 60502 is ready for pickup and the weather details are...\n\nIf there is any missing information from the tool you just respond back with the last available information from the tool."},
 {'role': 'user', 'content': 'What is the status of the order 1234599'},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [{'id': 'call_5itmdTYu72PJxibkOb4vyQz8',
    'type': 'function',
    'function': {'name': 'order_status',
     'arguments': '{"order_id":"1234599"}'}}]},
 {'role': 'tool',
  'content': 'Order status for 1234599 at zip 60502 is ready f

In [243]:
completion_2 = client2.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools
)

print(completion_2.choices[0].message)

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_z6WnFZwalaGtQzZyCi9bH2j6', function=Function(arguments='{"zip_code":"60502"}', name='weather_call'), type='function')], annotations=[])


In [249]:
completion_2

ChatCompletion(id='chatcmpl-BA1OZXQWE0XkywHsiktsMTx70ASVH', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_z6WnFZwalaGtQzZyCi9bH2j6', function=Function(arguments='{"zip_code":"60502"}', name='weather_call'), type='function')], annotations=[]))], created=1741728343, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_eb9dce56a8', usage=CompletionUsage(completion_tokens=17, prompt_tokens=247, total_tokens=264, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

**Now as you can see above we are getting a tool_call to be made for zip**

In [244]:
tool_call = completion_2.choices[0].message.tool_calls[0]
args = json.loads(tool_call.function.arguments)

In [245]:
zip_results = eval(tool_call.function.name)(**args)
zip_results

'Weather for 60502 is sunny today if you want to go to the store'

In [246]:
assistant_dict = get_tool_response(completion_2)
assistant_dict

{'role': 'assistant',
 'content': None,
 'tool_calls': [{'id': 'call_z6WnFZwalaGtQzZyCi9bH2j6',
   'type': 'function',
   'function': {'name': 'weather_call', 'arguments': '{"zip_code":"60502"}'}}]}

In [247]:
messages.append(assistant_dict)
messages

[{'role': 'system',
  'content': "You are retial ecom bot agent. You are capabale of giving order status info along with weather based on the order zip availability. \nOrder Zip will be returned by order_status and not consider the zip code provided by users.\nAny other request you get you politely says sorry and can't be processed\n\nYour available function are:\n\norder_status\n\nweather_call\n\nYou then output:\nOrder status for 12345 at zip 60502 is ready for pickup and the weather details are...\n\nIf there is any missing information from the tool you just respond back with the last available information from the tool."},
 {'role': 'user', 'content': 'What is the status of the order 1234599'},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [{'id': 'call_5itmdTYu72PJxibkOb4vyQz8',
    'type': 'function',
    'function': {'name': 'order_status',
     'arguments': '{"order_id":"1234599"}'}}]},
 {'role': 'tool',
  'content': 'Order status for 1234599 at zip 60502 is ready f

In [253]:
# messages.pop()

In [254]:
messages.append(
    {"role": "tool", "content": zip_results, "tool_call_id": tool_call.id}
)

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Order status for 1234599 at zip 60502 is ready for pickup and the weather details are sunny if you want to go to the store.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, annotations=[])


In [255]:
print(completion.choices[0].message.content)

Order status for 1234599 at zip 60502 is ready for pickup and the weather details are sunny if you want to go to the store.
